In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('/Users/dabeen/Desktop/K-DigitalTraining/Project/Machine_Learning/Final_Project/test_08.csv')
ratings

,title,user_id,user_name,reple_score,text
0,장민호 드라마 최종회,17923077,은하수(kys6****),10,콘서트에서 느낀 감정과 또 다르게 감동적이었습니다 외모 연기 가창력짙은 호소력 어느...
1,장민호 드라마 최종회,17923103,09s(09st****),10,"멋진 외모에 안구정화, 감성 깊은 노래에 귀정화최고의 감동이예요^^"
2,장민호 드라마 최종회,17923073,폴에버리치(opch****),10,노래할 때 세밀한 표정 가 볼 수 있어 너무 좋았어요 여기 최고의 가수
3,장민호 드라마 최종회,17923060,라나(8794****),10,재미와 감동이있습니다
4,장민호 드라마 최종회,17923054,현서(jizw****),10,너무 재밌었어요!! 제가 아는 노래가 많이 나와서 엄청 좋았어요!! 엄마한테 겨우 ...
...,...,...,...,...,...
199895,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,13447335,KH(hwal****),10,한국영화를 좋아합니다. 그중에 청연과 홍반장이 있었는데 하.. 당신의 그 연기같지않...
199896,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,13690895,psm2****,8,내가 좋아하는배우 김주혁 지금은 고인이 되었지만 제목이 유치해서 영화마저 유치하다 ...
199897,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,17756451,도로시짱(rest****),10,"스토리, 연기 다 좋아요드라마 나와서 다시 봐도 좋음 김주혁 엄정화가 주는 감동....."
199898,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,17691152,uchi****,10,정말 웃겼음 웃으면서 기분좋은 영화 유쾌하게


In [3]:
ratings = ratings.drop(['user_name', 'text'], axis=1)
ratings

,title,user_id,reple_score
0,장민호 드라마 최종회,17923077,10
1,장민호 드라마 최종회,17923103,10
2,장민호 드라마 최종회,17923073,10
3,장민호 드라마 최종회,17923060,10
4,장민호 드라마 최종회,17923054,10
...,...,...,...
199895,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,13447335,10
199896,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,13690895,8
199897,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,17756451,10
199898,어디선가 누군가에 무슨 일이 생기면 틀림없이 나타난다 홍반장,17691152,10


In [ ]:
# 완전 실패
# 다른방법 츄라이 해보기

user_ratings = ratings.pivot_table(index=['user_id'], columns=['title'], values='reple_score')
# user_ratings = user_ratings.dropna(thresh=10, axis=1).fillna(0)
user_ratings = user_ratings.fillna(0)
item_similarity_df = user_ratings.corr(method='pearson')

def get_similar_movies(movie_name, user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

action_lover = [("라라랜드", 10), ("킹메이커", 5), ("돈 룩 업", 2)]
similar_movies = pd.DataFrame()

for movie, rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie, rating), ignore_index=True)

similar_movies.head()
similar_movies.sum().sort_values(ascending=False)


In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
# Movie recommender with using cosine similarity score

df = pd.read_csv('/Users/dabeen/Desktop/K-DigitalTraining/Project/Machine_Learning/Final_Project/result_data.csv')
df['Movie_id'] = range(1, len(df)+1)

columns = ['Movie_id', 'Title', 'netizen', 'genre', 'director', 'actor', 'summary']

# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
    important_features = []
    for i in range(0, data.shape[0]):
        important_features.append(str(data['actor'][i])+' '+str(data['director'][i])+' '+str(data['genre'][i])+' '+str(data['Title'][i])+' '+str(data['summary'][i]))
    return important_features

# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

# Conver the text to a matrix of token counts
cm = CountVectorizer().fit_transform(df['important_features'])

# Get the cosine similarity matrix from teh count matrix
cs = cosine_similarity(cm)

# Get the title of the movie that the user likes
title = '알라딘'

# Find the movies id
movie_id = df[df.Title == title]['Movie_id'].values[0]

# Create a list of enumerations for the similarity score [ (movie_id, similarity score), (...) ]
scores = list(enumerate(cs[movie_id]))

# Sort the list
sorted_scores = sorted(scores, key = lambda x : x[1], reverse=True)
sorted_scores = sorted_scores[1:]

# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended movies to', title, 'are:\n')
for item in sorted_scores:
    movie_title = df[df.Movie_id == item[0]]['Title'].values[0]
    print(j+1, movie_title)
    j = j+1
    if j>6:
        break

The 7 most recommended movies to 알라딘 are:

1 인어 공주
2 내부자들: 디 오리지널
3 씽2게더
4 장화, 홍련
5 게임의 법칙
6 로마
7 터미널
